# Link Prediction : 7가지 = 동일한 커뮤니티 (그래프 전체), 다른 커뮤니티 (서로 다른 서브그래프들)
- [참고] https://brain-nim.tistory.com/m/76  

## 1. 하나의(동일한) community에서 링크를 예측하는 휴리스틱한 방법들 : Heuristics = CN, Jaccard, RA, AA, PA
- CN 에서 거리 2인 경로 구하는 과정만 추가되고, 이후의 계산은 모두 `networkx` 라이브러리 사용  
  
1. `CN (Common Neighbors)` : $CN(x,y)=|N(x)\cap N(y)|$  
  
2. `Jaccard Coefficient` : $J(x,y)=\frac{|N(x)\cap N(y)|}{|N(x)\cup N(y)|}$  
  
3. `RA (Resource Allocation)` : $RA(x,y)=\sum_{v\in N(x)\cap N(y)}\frac{1}{|N(v)|}$  
  
4. `AA (Adamic-Adar Index)` : $AA(x,y)=\sum_{v\in N(x)\cap N(y)}\frac{1}{\log |N(v)|}$  
  
5. `PA (Preferential Attachment Model, Barabasi Albert Model)` : $PA(x,y)=|N(x)||N(y)|$  

### 1-0. 예제 그래프 만들기 : 하나의 커뮤니티만 다루기 때문에 그래프 전체를 생각 
- $G=(V,E),~~|V|=9,~~ |E|=12$  
  
- 나중에 Cora dataset 으로 만든 그래프 넣어서 실험하면 됨

In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch_geometric.datasets import Planetoid#, CoraFull
import torch_geometric.transforms as T
import networkx as nx
import warnings
warnings.filterwarnings(action='ignore')
## Load Dataset
# load the Cora dataset
dataset = 'Cora'
path = os.path.join('../', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]
print(dataset.data)
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Has isolated nodes: {data.has_isolated_nodes()} -> i.e., no nodes not connected by edges')
print(f'Has self-loops: {data.has_self_loops()} -> i.e., no self-loops')
print(f'Is undirected: {data.is_undirected()} -> i.e., not directional')


In [ ]:
edge_index = data.edge_index.numpy()
print(edge_index.shape)
edge_example = edge_index[:, np.where(edge_index[0]==30)[0]]
print(edge_example)

node_example = np.unique(edge_example.flatten())
plt.figure(figsize=(10, 6))
G = nx.Graph()
G.add_nodes_from(node_example)
G.add_edges_from(list(zip(edge_example[0], edge_example[1])))
nx.draw_networkx(G, with_labels=False)

In [ ]:
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')

In [ ]:
import pandas as pd
from torch_geometric.utils import to_networkx

G = to_networkx(data, to_undirected=True)
degrees = [val for (node, val) in G.degree()]
display(pd.DataFrame(pd.Series(degrees).describe()).transpose().round(2))
print(len(degrees))
print(sum(degrees))
plt.figure(figsize=(10, 6))
plt.hist(degrees, bins=50)
plt.xlabel("node degree")
plt.show()

In [ ]:
## Cora data nx.draw
G = to_networkx(data, to_undirected=True)
pos = nx.spring_layout(G, seed=42)
cent = nx.degree_centrality(G)
node_size = list(map(lambda x: x * 500, cent.values()))
cent_array = np.array(list(cent.values()))
threshold = sorted(cent_array, reverse=True)[10]
print("threshold", threshold)
cent_bin = np.where(cent_array >= threshold, 1, 0.1)
plt.figure(figsize=(12, 12))
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
                               cmap=plt.cm.plasma,
                               node_color=cent_bin,
                               nodelist=list(cent.keys()),
                               alpha=cent_bin)
edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3)
plt.show()

### 1-1. `CN (Common Neighbors)`  
- $CN(x,y)=|N(x)\cap N(y)|$  
    where $x,y$ : nodes and $N(x), N(y)$ : neighborhoods of $x,y$ (respectively)  
  
- [예] $CN(A,C)=|\{B,D\}|=2$  
  
- 알고리즘 순서  
    a. 비연결 node pair 구하기  
  
    b. 거리 2인 경로 구하기  
    - 방법 : a 에서 구한 각 비연결 노드 pair에 대한 (1-hop) 이웃 노드를 거치면 거리 2인 경로  
    - `nx.common_neighbors(G, u, v)` where $G$ : undirected graph and $u,v$ : nodes  
    - https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.common_neighbors.html
  
    c. 각 node pair 마다 길이 2인 경로의 개수 세기

In [ ]:
## (1) CN (Common Neighbors) 
#### 엣지 연결이 없는 노드쌍
non_edges = list(nx.non_edges(G))
print(non_edges)

In [ ]:
#### 길이 2인 경로 구하기 : 
CN = {}
for s,t in non_edges:
    print(s,t, list(nx.common_neighbors(G,s,t))) 
## A-C는 경로 2개라는 거임 -> 길이 2인 경로가 가장 많음 

In [ ]:
## (1) CN (Common Neighbors) 
# 엣지 연결이 없는 노드쌍
non_edges = list(nx.non_edges(G))
# print(non_edges)

# 공통이웃 수 count 딕셔너리 생성
common_neighbors_count = {}
for n1,n2 in non_edges:
    common_neighbors_count[(n1, n2)] = len(list(nx.common_neighbors(G, n1, n2)))
    # print(common_neighbors_count)
sorted(common_neighbors_count.items(), key=lambda x: x[1], reverse=True)

### 1-2. `Jaccard Coefficient (- index, - similarity)`  
- $J(x,y)=\frac{|N(x)\cap N(y)|}{|N(x)\cup N(y)|}$  
  
- [예] $J(A,C)=\frac{|\{B,D\}|}{|\{B,D,E,F\}|}=\frac{2}{4}=0.5$  
  
- `nx.jaccard_coefficient(G, ebunch=None)`  where `ebunch` : iterable of node pairs, optional (default = None)  
  
- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.jaccard_coefficient.html#networkx.algorithms.link_prediction.jaccard_coefficient 

In [ ]:
## (2) Jaccard
jaccard = list(nx.jaccard_coefficient(G))
print('unsorted jaccard index :\n%s' % jaccard)
jaccard = sorted(jaccard, key=lambda x: x[2], reverse=True)  
print('---------------------------------------------------')
# jaccard.sort(key=lambda x: x[2], reverse=True)  
# print('Jaccard = %s' % jaccard)
for u,v,j in jaccard:
    print(f"({u},{v}) -> {j}")#:.8f}")

### 1-3. `RA, Resource Allocation (자원할당)`  
- $RA(x,y)=\sum_{v\in N(x)\cap N(y)}\frac{1}{|N(v)|}$  
  
- [예] $RA(A,C)=\frac{1}{|N(B)|}+\frac{1}{|N(D)|}=\frac{1}{3}+\frac{1}{3}=\frac{2}{3}=0.666\ldots$  
    - $N(A)\cap N(C)=\{B,D,E\}\cap \{B,D,F\}=\{B,D\}$
  
- `resource_allocation_index(G, ebunch=None)`
  
- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.resource_allocation_index.html

In [ ]:
## (3) RA, 자원 할당
resource = list(nx.resource_allocation_index(G))
resource = sorted(resource, key=lambda x: x[2], reverse=True)
for u,v,r in resource:
    print(f"({u},{v}) -> {r}")

### 1-4. `AA, Adamic Adar`  
- $AA(x,y)=\sum_{v\in N(x)\cap N(y)}\frac{1}{\log|N(v)|}$  
  
- [예] $RA(A,C)=\frac{1}{\log|N(B)|}+\frac{1}{\log|N(D)|}=\frac{1}{\log 3}+\frac{1}{\log 3}=1.820478\ldots$  
  
- `resource_allocation_index(G, ebunch=None)`  
  
- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.resource_allocation_index.html#networkx.algorithms.link_prediction.resource_allocation_index

In [ ]:
## AA
aa = list(nx.adamic_adar_index(G))
aa = sorted(aa, key=lambda x: x[2], reverse=True)
for u,v,a in aa:
    print(f"({u},{v}) -> {a}")

### 1-5. `PA, Preferential Attachment, Barabasi Albert`
- $PA(x,y)=|N(x)||N(y)|$  
  
- [예] $PA(A,C)=|N(A)||N(C)|=3\times 3=9$  
  
- `preferential_attachment(G, ebunch=None)`  
  
- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.preferential_attachment.html#networkx.algorithms.link_prediction.preferential_attachment

In [ ]:
## (5) PA 
pref = list(nx.preferential_attachment(G))
pref = sorted(pref, key=lambda x: x[2], reverse=True)
for u,v,p in pref:
    print(f"({u},{v}) -> {p}")

## 2. 서로 다른 커뮤니티에서의 링크 예측 
- `networkx` 라이브러리 이용해서 계산하면 되므로 그래프만 만들면 됨
### 2-0. 예제 그래프 만들기 : 하나의 그래프에서 서로 다른 두 커뮤니티(즉, 두 부분그래프)를 다룰 것임 -> 위와 같은 그래프인데, 두 서브그래프로 나눠서 다룸 
- `community0 = [A,B,C,D]`  
- `community1 = [E,F,G,H,I]`

In [ ]:
# community0 = ['A','B','C','D']
# community1 = ['E','F','G','H','I']
# labels = dict(zip(list(range(8)), community0 + community1))
# # print(list(range(0,4)), list(range(4,9)))
# print(labels)

- node coloring 참고 : https://choiseokwon.tistory.com/172

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# G = nx.Graph()
# G.add_edge(1,2, weight=5)
# G.add_edge(1,3, weight=6)
# G.add_edge(1,4, weight=2)
# G.add_edge(2,4, weight=1)
# G.add_edge(1,4, weight=0.5)
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True)
# labels = nx.get_edge_attributes(G, 'weight')
# print(labels)
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

In [ ]:
import networkx as nx

edges = [('A','B'),('A','D'),('A','E'),('B','D'),('B','C'),('C','D'),('C','F'),
        ('E','F'),('E','G'),('F','G'),('G','H'),('G','I')]
community0 = ['A','B','C','D'] ## 빨간색 노드
community1 = ['E','F','G','H','I'] ## 파란색 노드

G = nx.Graph()
G.add_edges_from(edges) 
pos = nx.spring_layout(G, seed=2023) ## positions for all nodes 
nx.draw(G, pos=pos, with_labels=True, font_color="whitesmoke") 
nx.draw_networkx_nodes(G, pos, ['A','B','C','D'], node_color="tab:red")
nx.draw_networkx_nodes(G, pos, ['E','F','G','H','I'], node_color="tab:blue")
G.add_nodes_from(community0, community=0)
G.add_nodes_from(community1, community=1)
G.nodes(data=True)

### 2-1. `Community CN (C.N. Soundarajan-Hopcroft score)`
- "the common neighbors Soundarajan-Hopcroft score of nodes $x$ and $y$"
    - $CCN(x,y):=|N(x)\cap N(y)|+\sum_{v\in N(x)\cap N(y)}f(v)$, where $f(v)= 1$ if $v\in N(x)\cap N(y)$ and 0, otherwise  
  
    - 공통 이웃의 수 $=|N(x)\cap N(y)| = CN(x,y)$  
  
    - 공통 이웃이 같은 커뮤니티에 속하는지의 여부 $=\sum_{v\in N(x)\cap N(y)}f(v)$ 
  
- [예1] $CCN(A,C)=CN(A,C)+\sum_{v\in\{B,D\}}f(v)=|\{B,D\}|+(f(B)+f(D))=2+(1+1)=4$  
  
- [예2] $CCN(F,I)=CN(F,I)+\sum_{v\in\{G\}}f(v)=|\{G\}|+f(G)=1+1=2$  
  
- `cn_soundarajan_hopcroft(G, ebunch=None, community='community')`  
  
- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.cn_soundarajan_hopcroft.html

In [ ]:
cn_s_h = list(nx.cn_soundarajan_hopcroft(G))
cn_s_h = sorted(cn_s_h, key=lambda x: x[2], reverse=True)
for u,v,csh in cn_s_h:
    print(f"({u}, {v}) -> {csh}")

### 2-2. `Community RA (Community Resource Allocation)`
- "the Resource Allocation Soundarajan-Hopcroft score of nodes $x$ and $y$"
    - $CRA(x,y):=\sum_{v\in N(x)\cap N(y)}\frac{f(v)}{|N(v)|}$, where $f(v)= 1$ if $v\in N(x)\cap N(y)$ and 0, otherwise  
  
    - 기존 RA 와 비슷하나, 분자에 1 대신 $f(v)$ -> 커뮤니티에 속했는지의 여부도 헤아림 
  
- [예1] $CRA(A,C)=\sum_{v\in\{B,D\}}\frac{f(v)}{|N(v)|}=\frac{f(B)}{|N(B)|}+\frac{f(D)}{|N(D)|}=\frac{1}{3}+\frac{1}{3}=\frac{2}{3}=0.666\ldots$  
  
- [예2] $CRA(F,I)=\sum_{v\in\{G\}}\frac{f(v)}{|N(v)|}=\frac{1}{4}=0.25$  
  
- `ra_index_soundarajan_hopcroft(G, ebunch=None, community='community')`  
  
- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.ra_index_soundarajan_hopcroft.html#networkx.algorithms.link_prediction.ra_index_soundarajan_hopcroft

In [ ]:
ra_s_h = list(nx.ra_index_soundarajan_hopcroft(G))
ra_s_h = sorted(ra_s_h, key=lambda x: x[2], reverse=True)
for u,v,rsh in ra_s_h:
    print(f"({u}, {v}) -> {rsh}")